# Stage 1: Generate Stock Universe

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Gather price histories

In [8]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---5311'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

In [3]:
td_ameritrade = amc.AmeritradeRest(username, password, client_id, chrome_executabel_path)
td_ameritrade.authenticate()

if len(td_ameritrade.authorization) == 0:
    print('Error: No authorization data: {}'.format(td_ameritrade.authorization))
else:
    print('You have authorization')

You have authorization


## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

## First lets see why stocks we already own for a specific account

I only want to work with Equity investments. This is kind of confusing, but at the account level assets that can be traded are call "EQUITY". When you get quotes for each asset, the same asset can be something like "ETF".

I also use Ameritrade's portfolio planner tool to create an asset mix based off of their reccomendations. I don't want these stocks (or in my case mutual funds and ETFs) to be part of this analysis. So I'll remove them here.

In [4]:
# Specific Portfolio Account
account_portfolio_df = utils.get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
equity_investments_df = utils.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')

# Filter out non Equity investments
current_stocks = amc.AmeritradeRest(username, password, client_id).get_quotes(utils.get_investment_symbols(equity_investments_df)).query('assetType == "EQUITY"').index.tolist()
current_investments_df = equity_investments_df[equity_investments_df['symbol'].isin(current_stocks)]
current_investments_df

,account,shortQuantity,averagePrice,currentDayProfitLoss,currentDayProfitLossPercentage,longQuantity,settledLongQuantity,settledShortQuantity,marketValue,maintenanceRequirement,currentDayCost,previousSessionLongQuantity,assetType,cusip,symbol,description,type
12,#---5311,0.0,21.62526,15.6600,0.12,783.0,783.0,0.0,12723.75,12723.75,0.0,783.0,EQUITY,88688T100,TLRY,0,0
19,#---5311,0.0,0.14474,-303.7500,-3.41,45000.0,45000.0,0.0,8608.50,0.00,0.0,45000.0,EQUITY,Q3860H107,FGPHF,0,0
20,#---5311,0.0,13.74495,1150.1000,4.61,2000.0,1000.0,0.0,26100.00,26100.00,12439.9,1000.0,EQUITY,98138J206,WKHS,0,0
21,#---5311,0.0,0.04960,-0.3375,-5.03,250.0,250.0,0.0,6.38,0.00,0.0,250.0,EQUITY,Q3394D101,EEENF,0,0
22,#---5311,0.0,59.22000,-35.0000,-1.44,50.0,50.0,0.0,2402.50,2402.50,0.0,50.0,EQUITY,26142R104,DKNG,0,0


## Remove other assets

There may be some stocks that you are speculating on and do not want to be part of the analysis. Being a conservative investor, I have a percentage of my active portfolio (that is not part of the portfolio planner), that I have personally speculated on and are using for a long term play. These stocks will not be part of the portfolio optimization.

In [5]:
speculative_stocks = ['FGPHF', 'EEENF']
final_investments_df = current_investments_df[~current_investments_df['symbol'].isin(speculative_stocks)]
final_existing_stocks = utils.get_investment_symbols(final_investments_df)
final_existing_stocks 

['TLRY', 'WKHS', 'DKNG']

In [6]:
symbols_of_interest = ['MGM', 'PDYPF', 'NNXPF', 'WKHS']
# Hardcoded for now
symbols_via_specific_criteria = ['AAPL',
                                 'MSFT',
                                 'GOOG',
                                 'TSLA',
                                 'COKE',
                                 'IBM',
                                 'BABA',
                                 'GMGMF',
                                 'OEG',
                                 'LX',
                                 'AIH',
                                 'NMRD',
                                 'CAN',
                                 'MOSY',
                                 'QFIN',
                                 'OCG',
                                 'PRTK',
                                 'ZKIN', 
                                 'EFOI',
                                 'CONN',
                                 'LEDS',
                                 'TELL',
                                 'JZXN',
                                 'VTNR',
                                 'AEI',
                                 'IPQ',
                                 'RCON'
                                ]

In [19]:
snp_500_df = utils.get_snp500()
display(snp_500_df.head())
snp_500_symbols = snp_500_df.index.to_list()
quote_dfs = []
for i in range(0, 500, 100):
    quote_dfs.append(amc.AmeritradeRest(username, password, client_id).get_quotes(snp_500_symbols[i:i+100]))
    
snp_500_quotes_df = pd.concat(quote_dfs, axis=0)
snp_500_quotes_df.describe()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago , Illinois",1964-03-31,0000001800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago , Illinois",2012-12-31,0001551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin , Ireland",2011-07-06,0001467373,1989


5


,bidPrice,bidSize,askPrice,askSize,lastPrice,lastSize,openPrice,highPrice,lowPrice,closePrice,...,divAmount,divYield,regularMarketLastPrice,regularMarketLastSize,regularMarketNetChange,regularMarketTradeTimeInLong,netPercentChangeInDouble,markChangeInDouble,markPercentChangeInDouble,regularMarketPercentChangeInDouble
count,500.000000,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.000000,5.000000e+02,500.000000,500.000000,500.000000,500.000000
mean,187.570800,316.20000,207.185680,332.800000,196.797750,77245.200000,195.994486,198.448267,194.240329,198.684640,...,1.770110,1.597520,196.784640,3863.602000,-1.900000,1.625782e+12,-1.045206,-1.891400,-1.045415,-1.065235
std,361.380421,801.77506,385.239415,723.000428,368.098705,144553.478177,367.513475,372.166403,363.799665,372.713611,...,1.937324,1.465425,368.284751,5761.866423,7.510589,4.922379e+06,1.173816,7.460303,1.154860,1.145508
min,0.100000,0.00000,11.710000,0.000000,11.390000,0.000000,11.450000,11.505000,11.360000,11.550000,...,0.000000,0.000000,11.390000,1.000000,-149.190000,1.625774e+12,-7.873800,-149.190000,-7.873800,-7.873800
25%,56.012500,100.00000,60.417500,100.000000,59.190000,0.000000,58.977500,59.610000,58.297500,59.677500,...,0.175000,0.195000,59.190000,1123.000000,-2.132500,1.625774e+12,-1.649725,-2.132500,-1.602750,-1.605350
50%,108.030000,100.00000,117.640000,100.000000,114.165000,100.000000,113.385000,114.840000,112.687500,114.570000,...,1.360000,1.340000,113.930000,2210.000000,-1.000000,1.625785e+12,-0.998800,-0.980000,-0.991400,-1.022000
75%,199.772500,300.00000,221.400000,300.000000,212.080000,109125.000000,210.677500,213.000000,209.972500,214.367500,...,2.640000,2.512500,212.080000,4343.500000,-0.287500,1.625785e+12,-0.330100,-0.260000,-0.330650,-0.360900
max,5026.000000,11100.00000,5499.000000,8200.000000,5053.020000,922800.000000,5129.000000,5140.529800,5039.330100,5202.210000,...,16.520000,7.420000,5053.020000,68205.000000,34.830000,1.625789e+12,4.122900,28.419900,3.861200,3.861200


In [23]:
snp_500_quotes_df.columns

Index(['assetType', 'assetMainType', 'cusip', 'symbol', 'description',
       'bidPrice', 'bidSize', 'bidId', 'askPrice', 'askSize', 'askId',
       'lastPrice', 'lastSize', 'lastId', 'openPrice', 'highPrice', 'lowPrice',
       'bidTick', 'closePrice', 'netChange', 'totalVolume', 'quoteTimeInLong',
       'tradeTimeInLong', 'mark', 'exchange', 'exchangeName', 'marginable',
       'shortable', 'volatility', 'digits', '52WkHigh', '52WkLow', 'nAV',
       'peRatio', 'divAmount', 'divYield', 'divDate', 'securityStatus',
       'regularMarketLastPrice', 'regularMarketLastSize',
       'regularMarketNetChange', 'regularMarketTradeTimeInLong',
       'netPercentChangeInDouble', 'markChangeInDouble',
       'markPercentChangeInDouble', 'regularMarketPercentChangeInDouble',
       'delayed', 'realtimeEntitled', 'market_cap'],
      dtype='object')

In [37]:
snp_500_quotes_df['avgPrice'] = (snp_500_quotes_df.highPrice + snp_500_quotes_df.lowPrice) / 2
snp_500_quotes_df['avgDollarVolume'] = snp_500_quotes_df.avgPrice * snp_500_quotes_df.totalVolume
snp_500_tickers = snp_500_quotes_df['avgDollarVolume'].sort_values()[-300:].index.to_list()

In [38]:
stock_universe = set(symbols_of_interest + symbols_via_specific_criteria + final_existing_stocks + snp_500_tickers)
holdings = utils.get_holdings(final_investments_df, stock_universe)['marketValue']
display(holdings)
utils.save_port_data(holdings.reset_index(), portfolio_file_name)

symbol
A       0.0
AAL     0.0
AAPL    0.0
ABBV    0.0
ABT     0.0
       ... 
WY      0.0
WYNN    0.0
XLNX    0.0
XOM     0.0
ZKIN    0.0
Name: marketValue, Length: 327, dtype: float64

## Portfolio weights

With the portfolio stocks and the additional stocks, show how each of them contribute to the portfolio. Later, once we produce an optimized portfolio, we can generate a report on how much stock to buy/sell based of what we already have.

In [39]:
holding_weights = utils.get_portfolio_weights(holdings)
display(holding_weights)

symbol
A       0.0
AAL     0.0
AAPL    0.0
ABBV    0.0
ABT     0.0
       ... 
WY      0.0
WYNN    0.0
XLNX    0.0
XOM     0.0
ZKIN    0.0
Length: 327, dtype: float64

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 3 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [40]:
number_of_years = 3
price_histories = td_ameritrade.get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Empty candle data for IPQ
No candle data for OCG
No candle data for AXP
No candle data for ICE
No candle data for QCOM
No candle data for APD
No candle data for VIAC
No candle data for WST
No candle data for CONN
No candle data for DIS
No candle data for RTX
No candle data for CPRT
No candle data for ALGN
No candle data for FAST
No candle data for AMGN
No candle data for CHTR
No candle data for SPG
No candle data for LX
No candle data for T
No candle data for PDYPF
No candle data for KMI
No candle data for MCHP
No candle data for SYK
No candle data for NKE
No candle data for FITB
No candle data for ATVI
No candle data for GOOGL
No candle data for TER
No candle data for ED
No candle data for AON
No candle data for SCHW
No candle data for IQV
No candle data for DLTR
No candle data for DVN
No candle data for PRTK
No candle data for DOW
No candle data for BMY
No candle data for EQIX
No candle data for ILMN
No candle data for AMD
No candle data for XLNX
No candle data for ABT
No candle data

In [41]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,29.45,29.460,29.0401,29.46,5561700,MGM,2018-07-09
6804,96.56,97.725,96.2600,96.96,2645986,LOW,2018-07-09
23100,82.01,82.740,81.1300,82.58,771476,QRVO,2018-07-09
54611,47.42,48.060,47.1700,47.77,1176425,NCLH,2018-07-09
34440,194.45,194.690,190.8700,192.75,15466403,BABA,2018-07-09


In [42]:
price_histories = utils.read_price_histories(price_histories_file_name)
close = utils.get_close_values(price_histories)
close.tail()

ticker,AAL,AAPL,ABBV,ACN,ADI,ADP,ADSK,AEI,AEP,AFL,...,WELL,WFC,WLTW,WM,WMB,WMT,WY,WYNN,XOM,ZKIN
date,,,,,,,,,,,,,,,,,,,,,
2021-07-01 00:00:00+00:00,21.51,137.27,114.26,300.00,169.17,199.81,293.70,5.31,85.260,54.00,...,83.73,45.41,231.00,141.17,26.80,139.32,34.33,122.40,63.26,4.23
2021-07-02 00:00:00+00:00,21.48,139.96,115.17,304.84,170.67,201.48,297.74,5.63,85.200,54.15,...,84.92,45.07,231.70,141.97,26.94,140.11,34.31,121.50,63.17,3.95
2021-07-06 00:00:00+00:00,21.01,142.02,115.73,305.40,168.32,201.11,295.60,5.25,85.240,53.27,...,85.14,43.49,229.95,142.77,26.42,139.94,33.97,116.90,61.37,3.93
2021-07-07 00:00:00+00:00,20.31,144.57,116.75,309.61,165.43,203.34,296.11,4.86,85.620,53.76,...,84.87,43.40,230.77,144.04,26.28,139.71,34.84,112.91,60.41,3.62
2021-07-08 00:00:00+00:00,20.39,143.16,116.60,309.15,163.91,201.86,294.47,4.86,85.365,52.33,...,84.83,42.33,224.15,142.17,26.24,139.59,35.04,111.66,60.38,3.66
